In [ ]:
import pygame
import random
import time

# 初始化pygame
pygame.init()

# 游戏常数
SCREEN_WIDTH = 800
SCREEN_HEIGHT = 600
CARD_WIDTH = 100
CARD_HEIGHT = 150
GRID_SIZE = 4  # 4x4的卡片网格
CARD_VALUES = [i for i in range(1, (GRID_SIZE * GRID_SIZE) // 2 + 1)] * 2  # 配对的卡片值

# 颜色定义
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
BLUE = (0, 0, 255)
GREEN = (0, 255, 0)
RED = (255, 0, 0)
CARD_COLOR = (50, 100, 150)
CARD_BACK_COLOR = (200, 50, 50)

# 设定游戏窗口
screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption("记忆配对游戏")
font = pygame.font.SysFont('Arial', 36)
small_font = pygame.font.SysFont('Arial', 24)

class Card:
    def __init__(self, value):
        self.value = value
        self.visible = False
        self.rect = pygame.Rect(0, 0, CARD_WIDTH, CARD_HEIGHT)
        self.matched = False
    
    def draw(self, x, y):
        self.rect.x = x
        self.rect.y = y
        if self.visible or self.matched:
            # 绘制卡片正面
            pygame.draw.rect(screen, CARD_COLOR, self.rect)
            text = font.render(str(self.value), True, WHITE)
            text_rect = text.get_rect(center=self.rect.center)
            screen.blit(text, text_rect)
        else:
            # 绘制卡片背面
            pygame.draw.rect(screen, CARD_BACK_COLOR, self.rect)
            pygame.draw.rect(screen, WHITE, self.rect, 2)

def create_card_grid():
    # 创建卡片并洗牌
    random.shuffle(CARD_VALUES)
    cards = [Card(value) for value in CARD_VALUES]
    
    # 计算卡片位置
    start_x = (SCREEN_WIDTH - (GRID_SIZE * (CARD_WIDTH + 10))) // 2
    start_y = (SCREEN_HEIGHT - (GRID_SIZE * (CARD_HEIGHT + 10))) // 2 + 50  # 为计时器留出空间
    
    card_grid = []
    for row in range(GRID_SIZE):
        card_row = []
        for col in range(GRID_SIZE):
            card = cards[row * GRID_SIZE + col]
            card.draw(start_x + col * (CARD_WIDTH + 10), start_y + row * (CARD_HEIGHT + 10))
            card_row.append(card)
        card_grid.append(card_row)
    
    return card_grid

def draw_game_info(steps, elapsed_time):
    # 绘制步数和时间
    steps_text = small_font.render(f"步数: {steps}", True, BLACK)
    time_text = small_font.render(f"时间: {elapsed_time//60}:{elapsed_time%60:02d}", True, BLACK)
    screen.blit(steps_text, (20, 20))
    screen.blit(time_text, (20, 50))

def main():
    running = True
    clock = pygame.time.Clock()
    
    card_grid = create_card_grid()
    selected_cards = []
    matched_cards = []
    waiting = False
    wait_time = 0
    steps = 0
    start_time = time.time()
    game_over = False
    
    while running:
        current_time = time.time() - start_time
        screen.fill(WHITE)
        
        # 处理事件
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            
            if event.type == pygame.MOUSEBUTTONDOWN and not waiting and not game_over:
                pos = pygame.mouse.get_pos()
                for row in card_grid:
                    for card in row:
                        if (card.rect.collidepoint(pos) and not card.matched 
                            and card not in selected_cards and len(selected_cards) < 2):
                            card.visible = True
                            selected_cards.append(card)
            
            # 重新开始游戏
            if event.type == pygame.KEYDOWN and event.key == pygame.K_r and game_over:
                return main()  # 递归重启游戏
        
        # 游戏逻辑
        if not game_over:
            if len(selected_cards) == 2 and not waiting:
                steps += 1
                if selected_cards[0].value == selected_cards[1].value:
                    for card in selected_cards:
                        card.matched = True
                    matched_cards.extend(selected_cards)
                    selected_cards = []
                else:
                    waiting = True
                    wait_time = pygame.time.get_ticks() + 1000  # 等待1秒
            
            # 检查是否需要隐藏不匹配的卡片
            if waiting and pygame.time.get_ticks() >= wait_time:
                for card in selected_cards:
                    card.visible = False
                selected_cards = []
                waiting = False
            
            # 检查游戏是否结束
            if len(matched_cards) == GRID_SIZE * GRID_SIZE:
                game_over = True
        
        # 绘制卡片
        for row in card_grid:
            for card in row:
                card.draw(card.rect.x, card.rect.y)
        
        # 绘制游戏信息
        draw_game_info(steps, int(current_time))
        
        # 游戏结束显示
        if game_over:
            text = font.render("恭喜你赢了! 按R键重新开始", True, GREEN)
            screen.blit(text, (SCREEN_WIDTH // 2 - text.get_width() // 2, 50))
        
        pygame.display.flip()
        clock.tick(60)
    
    pygame.quit()

if __name__ == "__main__":
    main()